## Setup libraries

In [1]:
from math import sqrt
import numpy as np
import pandas as pd
from pandasql import sqldf
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping

# Setup tables

In [2]:
Schivenoglia = read_csv("RisultatiSchivenoglia.csv")
Schivenoglia = Schivenoglia.loc[:, ["Date", "Ammonia", "PM10", "PM25",
                        "Wind_speed", "Wind_direction", "Temperature", "Rainfall"]]

print(Schivenoglia.shape)
Schivenoglia.head()



(2524, 8)


,Date,Ammonia,PM10,PM25,Wind_speed,Wind_direction,Temperature,Rainfall
0,2014-01-01,14.1,56.0,56.0,1.508213,260.0,4.766667,0.2
1,2014-01-02,19.4,56.0,55.0,0.390941,332.0,5.479167,2.0
2,2014-01-03,10.2,43.0,39.0,2.104680,267.0,5.766667,1.4
3,2014-01-04,8.6,37.0,33.0,0.949083,104.0,6.950000,5.8
4,2014-01-05,7.7,18.0,16.0,1.130128,248.0,7.541667,14.2


In [3]:
#Primo quadrante
Q1 = Schivenoglia.loc[((Schivenoglia.Wind_direction > 0) &
                 (Schivenoglia.Wind_direction < 90)), :]

a = Q1.shape[0]
arr1 = []
arr1 = [1 for i in range(a)]

del Q1['Wind_direction']
Q1["Wind_direction"] = arr1
columns_titles = ["index", "Date", "Ammonia", "PM10", "PM25",
                  "Wind_speed",  "Temperature", "Rainfall", "Wind_direction" ]
                  
Q1 = Q1.reindex(columns=columns_titles)
Q1 = Q1.drop("index", axis = 1)

#Secondo quadrante
Q2 = Schivenoglia.loc[((Schivenoglia.Wind_direction > 90) &
                 (Schivenoglia.Wind_direction < 180)), :]
a = Q2.shape[0]
arr2 = []
arr2 = [2 for i in range(a)]

del Q2['Wind_direction']
Q2["Wind_direction"] = arr2

#Terzo quadrante
Q3 = Schivenoglia.loc[((Schivenoglia.Wind_direction > 180) &
                 (Schivenoglia.Wind_direction < 270)), :]

a = Q3.shape[0]
arr3 = []
arr3 = [3 for i in range(a)]

del Q3['Wind_direction']
Q3["Wind_direction"] = arr3

#Quarto quadrante
Q4 = Schivenoglia.loc[((Schivenoglia.Wind_direction > 270) &
                 (Schivenoglia.Wind_direction < 360)), :]

a = Q4.shape[0]
arr4 = []
arr4 = [4 for i in range(a)]

del Q4['Wind_direction']
Q4["Wind_direction"] = arr4

Schivenoglia = sqldf('''
SELECT *
FROM Q1
UNION
SELECT *
FROM Q2
UNION
SELECT *
FROM Q3
UNION
SELECT *
FROM Q4
''')
#del Schivenoglia['Wind_direction']
Schivenoglia.rename(columns={'Wind_direction': 'Quadrante'}, inplace=True)

Schivenoglia

/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_9771/951932218.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q1["Wind_direction"] = arr1
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_9771/951932218.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q2["Wind_direction"] = arr2
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_9771/951932218.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

,Date,Ammonia,PM10,PM25,Wind_speed,Temperature,Rainfall,Quadrante
0,2014-01-01,14.1,56.0,56.0,1.508213,4.766667,0.2,3
1,2014-01-02,19.4,56.0,55.0,0.390941,5.479167,2.0,4
2,2014-01-03,10.2,43.0,39.0,2.104680,5.766667,1.4,3
3,2014-01-04,8.6,37.0,33.0,0.949083,6.950000,5.8,2
4,2014-01-05,7.7,18.0,16.0,1.130128,7.541667,14.2,3
...,...,...,...,...,...,...,...,...
2472,2020-12-27,NaN,23.0,21.0,0.519307,1.945833,0.0,4
2473,2020-12-28,NaN,18.0,14.0,1.756182,1.222222,7.0,4
2474,2020-12-29,5.0,33.0,32.0,1.019866,2.134722,0.4,1
2475,2020-12-30,5.9,52.0,48.0,2.195260,2.540278,0.0,3


In [4]:
Schivenoglia

,Date,Ammonia,PM10,PM25,Wind_speed,Temperature,Rainfall,Quadrante
0,2014-01-01,14.1,56.0,56.0,1.508213,4.766667,0.2,3
1,2014-01-02,19.4,56.0,55.0,0.390941,5.479167,2.0,4
2,2014-01-03,10.2,43.0,39.0,2.104680,5.766667,1.4,3
3,2014-01-04,8.6,37.0,33.0,0.949083,6.950000,5.8,2
4,2014-01-05,7.7,18.0,16.0,1.130128,7.541667,14.2,3
...,...,...,...,...,...,...,...,...
2472,2020-12-27,NaN,23.0,21.0,0.519307,1.945833,0.0,4
2473,2020-12-28,NaN,18.0,14.0,1.756182,1.222222,7.0,4
2474,2020-12-29,5.0,33.0,32.0,1.019866,2.134722,0.4,1
2475,2020-12-30,5.9,52.0,48.0,2.195260,2.540278,0.0,3


In [6]:
Schivenoglia.to_csv(
    '/Users/marcovinciguerra/Github/GitTesi/Python/Schivenoglia/QuadrantiSchivenoglia/QuadrantiSchivenoglia.csv')

